In [32]:
import pandas as pd
import json
from preprocessing import preprocess
from functools import reduce


In [3]:
df_train = pd.read_csv('datasets/train_radiomics_hipocamp.csv')
df_test = pd.read_csv('datasets/test_radiomics_hipocamp.csv')

In [4]:
df_train = preprocess(df_train,mode="all")

In [5]:
x,y = df_train
x_columns = list(x.columns)
x_columns.sort()
x_columns = list(map(lambda x:x.split("_"),x_columns))
x_columns

[['Age'],
 ['Sex'],
 ['diagnostics', 'Image-original', 'Maximum'],
 ['diagnostics', 'Image-original', 'Mean'],
 ['diagnostics', 'Mask-original', 'VoxelNum'],
 ['exponential', 'firstorder', '10Percentile'],
 ['exponential', 'firstorder', '90Percentile'],
 ['exponential', 'firstorder', 'Energy'],
 ['exponential', 'firstorder', 'Entropy'],
 ['exponential', 'firstorder', 'InterquartileRange'],
 ['exponential', 'firstorder', 'Kurtosis'],
 ['exponential', 'firstorder', 'Maximum'],
 ['exponential', 'firstorder', 'Mean'],
 ['exponential', 'firstorder', 'MeanAbsoluteDeviation'],
 ['exponential', 'firstorder', 'Median'],
 ['exponential', 'firstorder', 'Minimum'],
 ['exponential', 'firstorder', 'Range'],
 ['exponential', 'firstorder', 'RobustMeanAbsoluteDeviation'],
 ['exponential', 'firstorder', 'RootMeanSquared'],
 ['exponential', 'firstorder', 'Skewness'],
 ['exponential', 'firstorder', 'TotalEnergy'],
 ['exponential', 'firstorder', 'Uniformity'],
 ['exponential', 'firstorder', 'Variance'],
 [

In [36]:

def recursive_merge(dict1, dict2):
    """
    Recursively merge two dictionaries. Combines sub-dictionaries where both have the same key.
    """
    result = dict1.copy()  # Start with a shallow copy of the first dictionary
    for key, value in dict2.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = recursive_merge(result[key], value)
        else:
            result[key] = value
    return result

def gen_trie(l):
    if l == []:
        return dict()
    else:
        head = l[0]
        tail = l[1:]

        if head == []:
            return gen_trie(tail)
        else:
            h_head = head[0]
            h_tail = head[1:]
            rec = gen_trie(tail)
            return recursive_merge({h_head:gen_trie([h_tail])} ,rec)

cols_trie = gen_trie(x_columns)
with open("cols_trie.json","+w") as f:
    json.dump(cols_trie,f,indent=2,sort_keys=True)
def tree_size(tree):
    res = 0
    for v in tree.values():
        res += 1
        if v != {}:
            res += tree_size(v)
    return res

def n_leaves(tree):
    res = 0
    for v in tree.values():
        if v != {}:
            res += tree_size(v)
        else:
            res += 1
    return res

In [31]:
list(cols_trie.keys())
res = []
for k,v in cols_trie.items():
    res = res + [(k,n_leaves(v))]
res

[('Age', 0),
 ('Sex', 0),
 ('diagnostics', 3),
 ('exponential', 93),
 ('gradient', 93),
 ('lbp-2D', 44),
 ('lbp-3D-k', 93),
 ('lbp-3D-m1', 44),
 ('lbp-3D-m2', 46),
 ('log-sigma-1-0-mm-3D', 93),
 ('log-sigma-2-0-mm-3D', 93),
 ('log-sigma-3-0-mm-3D', 93),
 ('log-sigma-4-0-mm-3D', 93),
 ('log-sigma-5-0-mm-3D', 93),
 ('logarithm', 93),
 ('original', 107),
 ('square', 93),
 ('squareroot', 93),
 ('wavelet-HHH', 93),
 ('wavelet-HHL', 93),
 ('wavelet-HLH', 93),
 ('wavelet-HLL', 93),
 ('wavelet-LHH', 93),
 ('wavelet-LHL', 93),
 ('wavelet-LLH', 93),
 ('wavelet-LLL', 93)]